Starting from a 
* pre-trained model with a 
* pre-trained tokenizer

we perform finetuning on a negation task

In [1]:
%load_ext autoreload
%autoreload 2

import os 
import numpy as np
import random
from tqdm import tqdm

from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from torch import device, cuda, version

import apex

import dcc_splitter as splitter
import ner_training as trainer
import pandas as pd

import seaborn
import matplotlib.pyplot as plt

from collections import namedtuple

from transformers import AutoTokenizer, RobertaTokenizer, AutoModelForTokenClassification

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [3]:
device = device("cuda:0") if cuda.is_available() else device("cpu")

/home/koekiemonster/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
device

device(type='cpu')

In [5]:
dcc_dir = None
output_dir = None
skip_file = None
n_splits = 10
random_state = None
base_folder = "/media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels"
output_folder = "fine_tuned_token_classification"
mod_name = "robbert-v2-dutch-base" # "robbert-v2-dutch-base" # belabBERT_115k # bert-base-dutch
 

args = namedtuple
args.task = "negation" # experiencer, temporality
args.model_path = os.path.join(base_folder, mod_name)
args.model_type = "roberta" # bertje 
args.output_dir = os.path.join(base_folder, output_folder)
args.num_epochs = 4
args.eval_steps = 10 
args.lr = 5e-5
args.batch_size= 16
args.gradient_accumulation_steps=1
args.block_size = 512 # block size determines inclusion 
args.save_model=False
args.bio=True
args.do_eval=False
args.do_write=False
args.bootstrap=False
args.do_print_class_report=False

random.seed(77)

In [6]:
# args.block_size determines how many text snippets are  used for training, see ner_training.py lines 118--141
# obviously this is a code-design flaw that should be mended.
# the dataset loader should  include the id_begin_end in the output

In [7]:
# dcc-splitter for folds
dcc_splitter = splitter.DCCSplitter(dcc_dir, output_dir, skip_file, n_splits, random_state, write_to_file=False)
splits = dcc_splitter.split()

In [8]:
# load NER DCC set
dcc = pd.read_csv("../data/RobBERT/DCC_df.csv", 
                  sep="\t", 
                  skip_blank_lines=True, 
                  engine="python", 
                  encoding="latin-1",
                  on_bad_lines="warn", 
                  keep_default_na=False)

In [9]:
dcc.Negation.value_counts()

O             146395
NotNegated     15713
Negated         3017
Name: Negation, dtype: int64

In [10]:
dcc.loc[dcc.BIO!='O'][['Id', 'Begin', 'End']].apply(lambda x: "_".join(x), axis=1).nunique()

11882

In [11]:
Texts = dcc.groupby('Id').Word.apply(lambda x: " ".join(x))

In [12]:
tag_ids = {'negation':{'B-Negated':0,'B-NotNegated':1,'I-Negated':2,'I-NotNegated':3},
          'temporality':{'B-Recent':0,'B-Historical':1,'B-Hypothetical':2,'I-Recent':3,
                         'I-Historical':4,'I-Hypothetical':5},
          'experiencer':{'B-Patient':0,'B-Other':1,'I-Patient':2,'I-Other':3}}

tag2id = tag_ids[args.task]
tokenizer = AutoTokenizer.from_pretrained(args.model_path)

## Over all document sources

improvement: only output best model based on validation scores

In [13]:
# cycle through folds
scores = []
predlist = []
test_lists = []
loss_history = {}
for idx, fold in tqdm(enumerate(splits)):
    # re-init model for each fold, otherwise it keeps on training the same throughout all folds..
    token_model = AutoModelForTokenClassification.from_pretrained(args.model_path, num_labels = len(tag2id))
    
    train_list, test_list = fold['train'], fold['test']
    
    ## eval is optional (to gauge the best number of steps/epochs)
    eval_list = random.choices(train_list,k=int(len(train_list)/10)) if args.do_eval else []
    eval_dcc = dcc.loc[dcc.Id.isin(eval_list)]
    test_dcc = dcc.loc[dcc.Id.isin(test_list)]
    train_dcc = dcc.loc[(dcc.Id.isin(train_list)) & (~dcc.Id.isin(eval_list))]
    
    test_list = test_dcc.Id.tolist()
    eval_list = eval_dcc.Id.tolist()

    ###
    train_dataset = trainer.TextDatasetFromDataFrame(train_dcc, tokenizer, args) 
    test_dataset = trainer.TextDatasetFromDataFrame(test_dcc, tokenizer, args)
    eval_dataset = trainer.TextDatasetFromDataFrame(eval_dcc, tokenizer, args)
    
    args.do_print_class_report=False
    # Train on all document sources
    trained_model, eval_loss_history = trainer.train_model(model=token_model.to(device), 
                                                            tokenizer=tokenizer, 
                                                            train_dataset=train_dataset, 
                                                            eval_dataset=eval_dataset, 
                                                            tag2id=tag2id,
                                                            device=device, 
                                                            args=args,
                                                            max_grad_norm=1.0,
                                                            amp=False)
    args.do_print_class_report=True
    # Evaluate on all document sources
    f1, prec, rec, preds, truth, test_ids = trainer.eval_model(model=trained_model, 
                                       tokenizer=tokenizer, 
                                       eval_dataset=test_dataset, 
                                       tag2id=tag2id, 
                                       device=device, 
                                       args=args, 
                                       return_pred=True)
    
    loss_history[idx]=eval_loss_history
    
    #test_ids = ["_".join(t) for t in zip(test_dcc.Id, test_dcc.Begin, test_dcc.End)]
    scores.append({'fold': idx, 'f1': f1, 'precision': prec, 'recall': rec})
    predlist.append({'fold': idx, 'prediction': preds, 'truth': truth, 'ids': test_ids})
    test_lists.append(test_ids)
    

0it [00:00, ?it/s]Some weights of the model checkpoint at /media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels/robbert-v2-dutch-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint

RobertaConfig {
  "_name_or_path": "/media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels/robbert-v2-dutch-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

Epoch 1: 100%|████████████████████████████████| 168/168 [48:45<00:00, 17.42s/it]


Training finished, best model f = 0.000
F1: 0.856 


1it [1:03:31, 3811.85s/it]

...

RobertaConfig {
  "_name_or_path": "/media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels/robbert-v2-dutch-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

Epoch 1: 100%|████████████████████████████████| 168/168 [49:08<00:00, 17.55s/it]


Training finished, best model f = 0.000


2it [2:07:00, 3810.29s/it]

F1: 0.861 
              precision    recall  f1-score   support

     Negated       0.81      0.83      0.82       846
  NotNegated       0.84      0.89      0.87      4894

   micro avg       0.84      0.88      0.86      5740
   macro avg       0.83      0.86      0.84      5740
weighted avg       0.84      0.88      0.86      5740



In [62]:
predlist_prep = []
for foldnum, foldres in enumerate(predlist):
    ids = foldres['ids']
    for prs, trs, ids in zip(foldres['prediction'], foldres['truth'], foldres['ids']):
        for pr, tr, _id in zip(prs, trs, ids):
            tmp_dict={}
            if len(pr)==len(tr)==0:
                tmp_dict['fold'] = foldnum
                tmp_dict['entity_id'] = _id
                tmp_dict['label'] = "n/a"
                tmp_dict['robbert'] = "n/a"
            elif len(pr)>0:
                tmp_dict['fold'] = foldnum
                tmp_dict['entity_id'] = _id
                tmp_dict['label'] = tr
                tmp_dict['robbert'] = pr                
            else:
                raise ValueError("predictions are empty while truth is not")    
            predlist_prep.append(tmp_dict)
predlist_df = pd.DataFrame(predlist_prep)
predlist_df['bio_label'] = predlist_df['label'].str.replace(r"([BI])\-[A-z]+", "\\1", 
                                                        regex=True, case=True).str.strip()
predlist_df['bio_robbert'] = predlist_df['robbert'].str.replace(r"([BI])\-[A-z]+", "\\1", 
                                                        regex=True, case=True).str.strip()

bio_pred = predlist_df[['entity_id', 'bio_label', 'bio_robbert']]
predlist_df = predlist_df.loc[predlist_df.bio_label=='B']
predlist_df.drop(['bio_label', 'bio_robbert', 'fold'], axis=1, inplace=True)

In [63]:
predlist_df['label'] = predlist_df.label.map({'B-NotNegated': 'not negated', 'B-Negated': 'negated'})
predlist_df['robbert'] = predlist_df.robbert.map({'B-NotNegated': 'not negated', 'B-Negated': 'negated'})

In [64]:
predlist_df

,entity_id,label,robbert
0,DL1667_66_74,not negated,not negated
1,DL1667_151_160,not negated,negated
2,DL1667_207_213,not negated,not negated
3,DL1866_88_99,not negated,not negated
4,DL1716_40_51,not negated,not negated
...,...,...,...
18700,SP1522_407_417,not negated,not negated
18702,SP1591_39_50,not negated,not negated
18703,SP1591_146_156,not negated,not negated
18705,SP1877_21_31,negated,negated


In [66]:
test_scores = pd.DataFrame(scores)

In [67]:
if args.do_eval:
    dfl = []
    for i in range(2):
        df = pd.DataFrame(loss_history[i])
        df['fold']=i
        dfl.append(df)
    eval_history = pd.concat(dfl).reset_index()
    eval_history['step'] = eval_history['step'].astype(int)
    eval_history['fold'] = eval_history['fold'].astype(int)

    fig, ax = plt.subplots(ncols=3, figsize=(18,5))
    seaborn.lineplot(data=eval_history, x='step', y='f1', hue='epoch', ax=ax[0])
    seaborn.lineplot(data=eval_history, x='step', y='recall', hue='epoch', ax=ax[1])
    seaborn.lineplot(data=eval_history, x='step', y='precision', hue='epoch', ax=ax[2])

In [68]:
# micro-averaged scores
test_scores

,fold,f1,precision,recall
0,0,0.856088,0.831526,0.882145
1,1,0.860540,0.839576,0.882578


In [69]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [70]:
# B-NotNegated, B-Negated
labmap = {'B-NotNegated': False, 'B-Negated': True, 'I-NotNegated': False, 'I-Negated': True}

manual_scores = []
confusion_matrix = []
for i in range(len(predlist)):
    # accuracy over all documents, flattened
    _predlist = [(labmap[t], 'B' if 'B-' in t else 'I') for l in predlist[i]['prediction'] for t in l if len(l)>0]
    _truthlist = [(labmap[t], 'B' if 'B-' in t else 'I') for l in predlist[i]['truth'] for t in l if len(l)>0]

    tr_c = []
    pr_c = []
    tr_r = []
    pr_r = []

    b_truth = []
    b_pred = []
    for _t,_p in zip(_truthlist, _predlist):
        if _t[1]==_p[1]=='B':
            tr_c.append(_t[0])
            pr_c.append(_p[0])
        tr_r.append(_t[0])
        pr_r.append(_p[0])

        b_truth.append(_t[1]=='B')
        b_pred.append(_p[1]=='B')

    tr_c, pr_c, tr_r, pr_r = np.array(tr_c), np.array(pr_c), np.array(tr_r), np.array(pr_r)
    b_truth, b_pred = np.array(b_truth), np.array(b_pred)

    TN_c = np.sum((pr_c==tr_c) & (pr_c==False))
    TP_c = np.sum((pr_c==tr_c) & (pr_c==True))
    FP_c = np.sum((pr_c!=tr_c) & (pr_c==True))
    FN_c = np.sum((pr_c!=tr_c) & (pr_c==False))

    TN_r = np.sum((pr_r==tr_r) & (pr_r==False))
    TP_r = np.sum((pr_r==tr_r) & (pr_r==True))
    FP_r = np.sum((pr_r!=tr_r) & (pr_r==True))
    FN_r = np.sum((pr_r!=tr_r) & (pr_r==False))

    TN_b = np.sum((b_pred==b_truth) & (b_pred==False))
    TP_b = np.sum((b_pred==b_truth) & (b_pred==True))
    FP_b = np.sum((b_pred!=b_truth) & (b_pred==True))
    FN_b = np.sum((b_pred!=b_truth) & (b_pred==False))


    # micro
    f1 = f1_score(tr_r, pr_r, average='micro')
    precision = precision_score(tr_r, pr_r, average='micro')
    recall = recall_score(tr_r, pr_r, average='micro')
    manual_scores.append({'list': 'raw', 
                          'fold': i, 
                          'focus': 'micro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # macro
    f1 = f1_score(tr_r, pr_r, average='macro')
    precision = precision_score(tr_r, pr_r, average='macro')
    recall = recall_score(tr_r, pr_r, average='macro')
    manual_scores.append({'list': 'raw', 
                          'fold': i, 
                          'focus': 'macro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # Negated
    f1 = f1_score(tr_r, pr_r)
    precision = precision_score(tr_r, pr_r)
    recall = recall_score(tr_r, pr_r)
    manual_scores.append({'list': 'raw', 
                          'fold': i, 
                          'focus': 'negated', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # NotNegated
    f1 = f1_score(~tr_r, ~pr_r)
    precision = precision_score(~tr_r, ~pr_r)
    recall = recall_score(~tr_r, ~pr_r)
    manual_scores.append({'list': 'raw',
                          'fold': i,
                          'focus': 'notnegated', 
                          'f1': f1,
                          'precision': precision,
                          'recall': recall})
    
    confusion_matrix.append({'list': 'raw', 'fold': i, 'TN': TN_r, 'TP': TP_r, 'FN': FN_r, 'FP': FP_r})    
    ######################################
    # micro
    f1 = f1_score(tr_c, pr_c, average='micro')
    precision = precision_score(tr_c, pr_c, average='micro')
    recall = recall_score(tr_c, pr_c, average='micro')
    manual_scores.append({'list': 'clean', 
                          'fold': i, 
                          'focus': 'micro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # macro
    f1 = f1_score(tr_c, pr_c, average='macro')
    precision = precision_score(tr_c, pr_c, average='macro')
    recall = recall_score(tr_c, pr_c, average='macro')
    manual_scores.append({'list': 'clean', 
                          'fold': i, 
                          'focus': 'macro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # Negated
    f1 = f1_score(tr_c, pr_c)
    precision = precision_score(tr_c, pr_c)
    recall = recall_score(tr_c, pr_c)
    manual_scores.append({'list': 'clean', 
                          'fold': i, 
                          'focus': 'negated', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # NotNegated
    f1 = f1_score(~tr_c, ~pr_c)
    precision = precision_score(~tr_c, ~pr_c)
    recall = recall_score(~tr_c, ~pr_c)
    manual_scores.append({'list': 'clean',
                          'fold': i,
                          'focus': 'notnegated', 
                          'f1': f1,
                          'precision': precision,
                          'recall': recall})
    
    confusion_matrix.append({'list': 'clean', 'fold': i, 'TN': TN_c, 'TP': TP_c, 'FN': FN_c, 'FP': FP_c})    
    ######################################
    # micro
    f1 = f1_score(b_truth, b_pred, average='micro')
    precision = precision_score(b_truth, b_pred, average='micro')
    recall = recall_score(b_truth, b_pred, average='micro')
    manual_scores.append({'list': 'B_I', 
                          'fold': i, 
                          'focus': 'micro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # macro
    f1 = f1_score(b_truth, b_pred, average='macro')
    precision = precision_score(b_truth, b_pred, average='macro')
    recall = recall_score(b_truth, b_pred, average='macro')
    manual_scores.append({'list': 'B_I', 
                          'fold': i, 
                          'focus': 'macro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # Negated
    f1 = f1_score(b_truth, b_pred)
    precision = precision_score(b_truth, b_pred)
    recall = recall_score(b_truth, b_pred)
    manual_scores.append({'list': 'B_I', 
                          'fold': i, 
                          'focus': 'negated', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # NotNegated
    f1 = f1_score(~b_truth, ~b_pred)
    precision = precision_score(~b_truth, ~b_pred)
    recall = recall_score(~b_truth, ~b_pred)
    manual_scores.append({'list': 'B_I',
                          'fold': i,
                          'focus': 'notnegated', 
                          'f1': f1,
                          'precision': precision,
                          'recall': recall})
    
    confusion_matrix.append({'list': 'B_I', 'fold': i, 'TN': TN_b, 'TP': TP_b, 'FN': FN_b, 'FP': FP_b})    
    
manual_scores_df = pd.DataFrame(data=manual_scores)
confusion_matrix_df = pd.DataFrame(data=confusion_matrix)

In [71]:
manual_scores_df.groupby(['list', 'focus']).mean()

fold        f1  precision    recall
list  focus                                          
B_I   macro        0.5  0.931249   0.937388  0.926587
      micro        0.5  0.934947   0.934947  0.934947
      negated      0.5  0.947193   0.928068  0.967163
      notnegated   0.5  0.915306   0.946709  0.886011
clean macro        0.5  0.957296   0.967843  0.947453
      micro        0.5  0.978474   0.978474  0.978474
      negated      0.5  0.927223   0.952916  0.902893
      notnegated   0.5  0.987369   0.982770  0.992012
raw   macro        0.5  0.950238   0.961279  0.939999
      micro        0.5  0.973703   0.973703  0.973703
      negated      0.5  0.916067   0.943428  0.890254
      notnegated   0.5  0.984409   0.979131  0.989744

In [23]:
Texts['DL1932']

'Diagnose : status na glasverwonding in de linkerpols met uitval van de nervus ulnaris . Er bleek toen sprake van een doorsnijding van de arteria ulnaris , de nervus ulnaris , de flexor carpi ulnaris en van beide flexoren van de pink , allen aan de linkerzijde . Bij onderzoek is er sprake van een litteken aan de ulnovolaire zijde van de pols , verlengd naar de onderarm . Er is ook duidelijk sprake van een atrofie van de abductor pollicis met een krachtsvermindering tot 1 / 5 van de normale kracht . De abductor digiti quinti heeft ook een duidelijk krachtsverlies . Van de atrofie van de kleine handspieren is met name die van de adductor pollicis het meest opvallend .'

## Append to other results 

In [24]:
merged_result_file = '../results/merged_results.csv.gz'
results = pd.read_csv(merged_result_file)

In [25]:
entities = results.entity_id.str.replace(r"\_[0-9]+\_[0-9]+", "").unique()
len(entities)

/tmp/ipykernel_29247/2061256545.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  entities = results.entity_id.str.replace(r"\_[0-9]+\_[0-9]+", "").unique()


5365

In [102]:
sanity_check = predlist_df[['entity_id', 'label']].set_index('entity_id').join(results[['entity_id', 'label']].set_index('entity_id'),
                                                                how='inner',rsuffix='_or')

(sanity_check['label'] == sanity_check['label_or']).sum()==sanity_check.shape[0]

True

In [108]:
predlist_df.set_index('entity_id', inplace=True)
results.set_index('entity_id', inplace=True)

In [112]:
total_results = results.join(predlist_df[['robbert']], how='left')
total_results.to_csv("../results/merged_results_new.csv.gz", index=True, compression='gzip')

In [118]:
unanimous = total_results.loc[total_results.robbert.isna()==False,
                  ['label', 'bilstm_cv', 'rule_based', 'robbert']].apply(lambda x: x[0]==x[1]==x[2]==x[3], 
                                                                         axis=1)

In [169]:
def number_of_dissenters(x):
    return int(x[0] != x[1])+\
           int(x[0] != x[2])+\
           int(x[0] != x[3])+\
           int(x[0] != x[4])

In [170]:
dissenters = total_results.loc[total_results.robbert.isna()==False,['label', 'bilstm', 'bilstm_cv', 'rule_based', 'robbert']]\
                            .apply(number_of_dissenters, axis=1)
total_results.loc[total_results.robbert.isna()==False, 'dissenters'] = dissenters.astype(int)

In [173]:
total_results.loc[total_results.dissenters==4]

,category,label,bilstm,bilstm_cv,rule_based,robbert,dissenters
entity_id,,,,,,,
DL1775_454_461,DL,not negated,negated,negated,negated,negated,4.0
GP1191_121_127,GP,negated,not negated,not negated,not negated,not negated,4.0
GP1601_67_74,GP,negated,not negated,not negated,not negated,not negated,4.0
GP1729_162_169,GP,not negated,negated,negated,negated,negated,4.0
GP2796_56_63,GP,negated,not negated,not negated,not negated,not negated,4.0
RD1338_114_126,RD,not negated,negated,negated,negated,negated,4.0
RD2031_132_137,RD,not negated,negated,negated,negated,negated,4.0
RD2200_75_85,RD,not negated,negated,negated,negated,negated,4.0
RD2557_205_216,RD,not negated,negated,negated,negated,negated,4.0


In [174]:
Texts['DL1775']

'Vanaf oktober 1994 heb ik de behandeling van dit crushletsel van de rechter hand met luxaties op carpometacarpaal niveau en een fractuur van de basisphalanx van de wijsvinger en van metacarpale III overgenomen . Hij werd op 25 juli 1995 opnieuw geopereerd , waarbij de loop via het oude litteken werd opgelicht en van het surplus subcutaan vet werd ontdaan . Tenslotte werd ter hoogte van de aangegeven neuroomklachten geexploreerd , waarbij geen duidelijke neuroom aangetroffen werd . Er blijkt een ruptuur te zijn ontstaan van de extensor van de vijfde straal , zodat reexploratie noodzakelijk is op 29 juli 1995 .'